<a href="https://colab.research.google.com/github/beyg1/Q4/blob/main/Quiz%20Practice/Orchestration_Handoffs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uq openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.4/161.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.5/158.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from agents import Agent, Runner, OpenAIChatCompletionsModel, set_tracing_export_api_key, trace, function_tool
from openai import AsyncOpenAI
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
set_tracing_export_api_key(OPENAI_API_KEY)

Client = AsyncOpenAI(
    api_key = GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.5-flash-lite",
    openai_client= Client
)                                                                      #Setting up the environment

#Simple Handoff/ Agent Orchestration

In [ ]:
import asyncio

async def main():
  billing_agent = Agent(
      name = "Billing Agent",
      model = model,
  )

  refund_agent = Agent(
      name = "Refund Agent",
      model = model,
  )

  triage_agent = Agent(
      name = "Orchestrator Agent",
      model = model,
      handoffs=[refund_agent, billing_agent]
  )
         # Simple Handoff/ Agent Orchestration
  with trace("Orchestration"):
    res = await Runner.run(triage_agent,"What's my bill for last month?")
    print(res.final_output)
    print(res.last_agent.name)

if __name__ == "__main__":
  asyncio.run(main())

I can help you with that. To access your billing information, I need to verify your identity. Could you please provide me with your account number or the email address associated with your account?
Billing Agent


 # Manipulate Handoffs with (handoff) as Handoffs are basically just another tool call for Agents

In [ ]:
import asyncio
from agents import handoff

async def main():
  billing_agent = Agent(
      name = "Billing Agent",
      model = model,
  )

  refund_agent = Agent(
      name = "Refund Agent",
      model = model,
  )

  triage_agent = Agent(
      name = "Orchestrator Agent",
      model = model,
      handoffs=[handoff(agent=refund_agent, tool_name_override="RefundAgent", tool_description_override="Handles Refund"),
                       handoff(agent=billing_agent, tool_name_override="BillingAgent", tool_description_override="Handles Billing", is_enabled=False)]
  )                                             # when is_enabled false, the orchestrator wont even see the agent

  with trace("handoff"):
    res = await Runner.run(triage_agent,"Transfer me to Billing dept")
    print(res.final_output)
    print(res.last_agent.name)

if __name__ == "__main__":
  asyncio.run(main())

I can only help with refunds. Is there anything else?
Orchestrator Agent


#We can apply input filters in handoff to remove access of tools of main agent from the agents who are handoffed to

In [30]:
import asyncio
from agents import handoff, function_tool
from agents.extensions import handoff_filters

async def main():

  @function_tool
  async def refund_policy()->str:
    """Have all the policies for refund"""
    return "Refund Policy #1:  10% wil be charged for every refund "

  billing_agent = Agent(
      name = "Billing Agent",
      instructions="You are an helpful Billing agent. when handoffs happen to you, briefly introduce yourself  and solve user querry",
      model = model,
  )

  refund_agent = Agent(
      name = "Refund Agent",
      instructions="You are an helpful Refund agent. when handoffs happen to you, briefly introduce yourself  and solve user querry immediately",
      model = model,
      tools=[refund_policy]
  )

  triage_agent = Agent(
      name = "Orchestrator Agent",
      model = model,                           # handoff filter remove all tools of main agents that are available to handoffed agents
      handoffs=[handoff(agent=refund_agent, input_filter=handoff_filters.remove_all_tools),
                       handoff(agent=billing_agent, )]    # Tool call happens here because tool is given to handoffed agent and not to triage agent itself
  )                                                                       # Lets try again but pass the tool to main triage agent next time

  with trace("handoff_filters"):
    res = await Runner.run(triage_agent,"transfer me to refund agent because I want to know refund policies?")
    print(res.final_output)
    print(res.last_agent.name)

if __name__ == "__main__":
  asyncio.run(main())

I am an AI Refund Agent. I can help you with refund policies. According to our policy, a 10% fee will be charged for every refund.
Refund Agent


In [37]:
import asyncio
from agents import handoff, function_tool
from agents.extensions import handoff_filters

async def main():

  @function_tool
  async def refund_policy()->str:
    """Have all the policies for refund"""
    return "Refund Policy #1:  10% wil be charged for every refund "

  billing_agent = Agent(
      name = "Billing Agent",
      instructions="You are an helpful Billing agent. when handoffs happen to you, briefly introduce yourself  and solve user querry",
      model = model,
  )

  refund_agent = Agent(
      name = "Refund Agent",
      instructions="You are an helpful Refund agent. when handoffs happen to you, briefly introduce yourself  and solve user querry immediately. if you dont have access to refund policy just tell user",
      model = model,
  )

  triage_agent = Agent(
      name = "Orchestrator Agent",
      model = model,
      tools=[refund_policy],
      handoffs=[handoff(agent=refund_agent, input_filter=handoff_filters.remove_all_tools), #So the refund agent cant access refund_policy tool of orchestrator agent
                       handoff(agent=billing_agent, )]                                                                       # but orchestrator agent can and also billing_agent can access it too
  )

  with trace("handoff_filters"):
    res = await Runner.run(triage_agent,"What is the refund policy? & Can you transfer me to refund agent?")
    print(res.final_output)
    print(res.last_agent.name)

if __name__ == "__main__":
  asyncio.run(main())

Hello! I am your refund agent.

I do not have access to the refund policy. Would you like me to try and find it for you, or is there anything else I can help you with?
Refund Agent
